In [1]:
import numpy as np
import pandas as pd
import requests
import json

import matplotlib.pyplot as plt

import panel as pn
# hvplot.extension('matplotlib')
pn.extension('tabulator')
pn.extension(sizing_mode = 'stretch_width')

import hvplot.pandas

In [2]:
data_path = '../../../data/all-data/test_data.csv'
df = pd.read_csv(data_path)
data = df.to_dict('records')
# 'http://127.0.0.1:8081/predict'
req = requests.post(
    'https://intrusion-detection-classifier.herokuapp.com/predict', json=data)
y_pred = json.loads(json.loads(req.text)[0])
df['predicted_label'] = y_pred
req.close()

df

,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,predicted_label
0,55553,0.0,2,0,1134,0,567.0,567.0,567.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dos
1,55234,0.0,2,0,132,0,66.0,66.0,66.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
2,46381,0.0,2,0,1324,0,662.0,662.0,662.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dos
3,34354,0.0,2,0,276,0,138.0,138.0,138.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
4,65526,0.0,2,0,418,0,209.0,209.0,209.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
5,443,0.0,2,0,3028,0,1514.0,1514.0,1514.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
6,22922,0.0,2,0,132,0,66.0,66.0,66.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
7,55234,0.0,2,0,132,0,66.0,66.0,66.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
8,443,0.0,2,0,3028,0,1514.0,1514.0,1514.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
9,443,0.0,2,0,1006,0,503.0,503.0,503.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dos


In [3]:
idf = df.interactive()

In [ ]:
port_radio_button = pn.widgets.RadioButtonGroup(
    name='Radio Button Group', 
    options=df.destination_port.value_counts().index.tolist(),
    button_type='success')
port_radio_button

In [ ]:
df.groupby(['destination_port', 'predicted_label'])['flow_duration'].size().to_frame().reset_index()

In [4]:
tabel_pipline = (idf)

In [5]:
port_pipline = (
    idf
    .groupby(['destination_port', 'predicted_label'])['flow_duration'].size()
    .to_frame()
    .reset_index()
)

In [7]:
tabel = tabel_pipline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, sizing_mode='stretch_width')

In [ ]:
port_count = df.predicted_label.value_counts().to_frame().reset_index()
port_plot = port_pipline.hvplot(
    kind='bar',
    x = 'predicted_label',
    y = 'flow_duration',
    title='Detected Attacks Distribution')
port_plot = port_plot.relabel("ylabel='Custom x-label'").opts(ylabel='Label Count')
port_plot = port_plot.relabel("xlabel='Custom x-label'").opts(xlabel='Predicted Label')

In [ ]:
template = pn.template.FastListTemplate(
    title='Intrusion Detection Dashboard',
    # sidebar=[

    # ],
    main=[
        pn.Row(port_plot.panel(width=500), margin=(0,25)),
        pn.Row(tabel.panel(width=00))
    ],
    accent_base_color='#88d8b0',
    header_background='#88d8b0'
)

template.servable()

In [16]:
pn.serve(template)

Launching server at http://localhost:54447
